# QC-Py-17 - Sentiment Analysis pour le Trading

> **Exploiter le sentiment de marche pour generer des signaux alpha**
> Duree: 75 minutes | Niveau: Intermediaire-Avance | Python + QuantConnect

---

## Objectifs d'Apprentissage

A la fin de ce notebook, vous serez capable de :

1. Comprendre les **differents types de sentiment** et leurs sources
2. Utiliser **TextBlob** et **VADER** pour analyser le sentiment de textes financiers
3. Creer un **Custom Data provider** pour integrer des donnees de sentiment dans QuantConnect
4. Developper un **Alpha Model** base sur l'agregation du sentiment
5. Implementer une **strategie contrarienne** exploitant les extremes de sentiment
6. Combiner **sentiment et analyse technique** pour des signaux robustes
7. Construire une **strategie complete** de trading sur sentiment

## Prerequis

- Notebooks QC-Py-01 a QC-Py-13 completes (bases QuantConnect, Alpha Models)
- Notions de base en traitement du langage naturel (NLP)
- Comprehension des indicateurs techniques (SMA, RSI)

## Structure du Notebook

1. Introduction au Sentiment (15 min)
2. Analyse de Sentiment avec Python (25 min)
3. Integration dans QuantConnect (30 min)
4. Sentiment Contrarien (15 min)
5. Combinaison Sentiment + Technique (20 min)
6. Strategie Complete (20 min)

---

## Setup et Imports

In [ ]:
# Imports standards
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from typing import Dict, List, Optional
import json
import warnings
warnings.filterwarnings('ignore')

# Configuration matplotlib
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

print("Imports de base reussis")

In [ ]:
# Installation des bibliotheques de sentiment (si necessaire)
# Decommenter pour installer:
# !pip install textblob vaderSentiment

try:
    from textblob import TextBlob
    print("TextBlob importe avec succes")
except ImportError:
    print("TextBlob non installe. Executez: pip install textblob")

try:
    from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
    print("VADER importe avec succes")
except ImportError:
    print("VADER non installe. Executez: pip install vaderSentiment")

---

## Partie 1 : Introduction au Sentiment (15 min)

### 1.1 Qu'est-ce que le Sentiment de Marche?

Le **sentiment de marche** represente l'attitude collective des investisseurs envers un actif ou le marche en general. Il peut etre mesure de differentes manieres:

| Type de Sentiment | Source | Description |
|-------------------|--------|-------------|
| **Market Sentiment** | Prix, volumes, options | Derive de l'activite de trading |
| **News Sentiment** | Articles, communiques | Analyse du ton des nouvelles |
| **Social Sentiment** | Twitter, Reddit, forums | Opinions des investisseurs retail |
| **Analyst Sentiment** | Rapports, ratings | Recommandations professionnelles |
| **SEC Filings Sentiment** | 10-K, 10-Q, 8-K | Ton des documents officiels |

### Sources de Donnees de Sentiment

```
                    Sources de Sentiment
                           |
     +---------------------+---------------------+
     |                     |                     |
     v                     v                     v
+----------+         +-----------+        +------------+
|   News   |         |  Social   |        | Corporate  |
+----------+         +-----------+        +------------+
|Bloomberg |         | Twitter/X |        | SEC EDGAR  |
|Reuters   |         | Reddit    |        | Earnings   |
|NewsAPI   |         | StockTwits|        | Transcripts|
|Benzinga  |         | Seeking   |        | Press      |
+----------+         | Alpha     |        | Releases   |
                     +-----------+        +------------+
```

In [ ]:
# Tableau des sources de sentiment et leur utilisation

sentiment_sources = {
    'News (Bloomberg, Reuters)': {
        'latence': 'Faible (temps reel)',
        'fiabilite': 'Haute',
        'cout': 'Eleve (abonnement)',
        'utilisation': 'Trading institutionnel'
    },
    'NewsAPI (gratuit tier)': {
        'latence': 'Moyenne (15-30 min)',
        'fiabilite': 'Moyenne',
        'cout': 'Gratuit/Faible',
        'utilisation': 'Recherche, prototypage'
    },
    'Twitter/X': {
        'latence': 'Tres faible (temps reel)',
        'fiabilite': 'Faible (bruit eleve)',
        'cout': 'API payante',
        'utilisation': 'Sentiment retail, memes'
    },
    'Reddit (r/wallstreetbets)': {
        'latence': 'Faible',
        'fiabilite': 'Faible a moyenne',
        'cout': 'API gratuite',
        'utilisation': 'Sentiment retail, momentum'
    },
    'SEC Filings': {
        'latence': 'Elevee (trimestriel)',
        'fiabilite': 'Tres haute',
        'cout': 'Gratuit (EDGAR)',
        'utilisation': 'Analyse fondamentale'
    }
}

print("Sources de Sentiment pour le Trading:")
print("="*70)
for source, details in sentiment_sources.items():
    print(f"\n{source}:")
    for key, value in details.items():
        print(f"  - {key.capitalize()}: {value}")

### 1.2 Impact du Sentiment sur les Prix

Le sentiment peut agir comme:

| Role | Description | Strategie |
|------|-------------|----------|
| **Leading Indicator** | Le sentiment precede le mouvement de prix | Suivre le sentiment |
| **Lagging Indicator** | Le sentiment suit le mouvement de prix | Contrarien |
| **Confirmateur** | Le sentiment confirme la tendance | Filtre additionnel |

### Challenges de l'Analyse de Sentiment

1. **Bruit** : Beaucoup de donnees non-pertinentes
2. **Manipulation** : Pump & dump, fake news, bots
3. **Latence** : Le temps de traitement peut etre trop long
4. **Contexte** : Mots identiques, sens differents ("short" = vendre vs court)
5. **Sarcasme** : Difficile a detecter automatiquement

In [ ]:
# Exemples de challenges dans l'analyse de sentiment financier

examples = [
    {
        'text': "Apple is killing it!",
        'expected': 'Positif',
        'challenge': 'Argot: "killing it" = excellent'
    },
    {
        'text': "Tesla stock is on fire today",
        'expected': 'Positif',
        'challenge': 'Metaphore: "on fire" = en hausse'
    },
    {
        'text': "Sure, $GME to the moon... again.",
        'expected': 'Sarcasme/Negatif',
        'challenge': 'Sarcasme difficile a detecter'
    },
    {
        'text': "I'm shorting this garbage stock",
        'expected': 'Negatif',
        'challenge': '"Shorting" = action bearish, "garbage" = pejoratif'
    },
    {
        'text': "Revenue beat expectations but guidance disappoints",
        'expected': 'Mixte/Neutre',
        'challenge': 'Sentiments opposes dans la meme phrase'
    }
]

print("Challenges de l'Analyse de Sentiment Financier:")
print("="*70)
for i, ex in enumerate(examples, 1):
    print(f"\nExemple {i}:")
    print(f"  Texte: \"{ex['text']}\"")
    print(f"  Sentiment attendu: {ex['expected']}")
    print(f"  Challenge: {ex['challenge']}")

---

## Partie 2 : Analyse de Sentiment avec Python (25 min)

### 2.1 TextBlob pour Sentiment de Base (10 min)

**TextBlob** est une bibliotheque Python simple pour le traitement du langage naturel. Elle fournit:

- **Polarity** : Score de -1 (negatif) a +1 (positif)
- **Subjectivity** : Score de 0 (objectif) a 1 (subjectif)

TextBlob est base sur un dictionnaire de mots pre-annotes et utilise des regles pour calculer le sentiment.

In [ ]:
from textblob import TextBlob

def analyze_sentiment_textblob(text: str) -> Dict:
    """
    Analyse le sentiment d'un texte avec TextBlob.
    
    Args:
        text: Le texte a analyser
    
    Returns:
        Dict avec polarity et subjectivity
    """
    blob = TextBlob(text)
    return {
        'polarity': blob.sentiment.polarity,      # -1 (negatif) a +1 (positif)
        'subjectivity': blob.sentiment.subjectivity  # 0 (objectif) a 1 (subjectif)
    }

# Exemples de titres financiers
headlines = [
    "Apple reports record-breaking quarterly earnings, stock surges",
    "Tesla faces massive recalls, shares plummet in after-hours trading",
    "Federal Reserve maintains interest rates unchanged",
    "Amazon announces exciting new AI features, investors optimistic",
    "Bank of America warns of potential recession in 2025"
]

print("Analyse de Sentiment avec TextBlob:")
print("="*70)

for headline in headlines:
    result = analyze_sentiment_textblob(headline)
    
    # Interpretation
    if result['polarity'] > 0.1:
        sentiment = "POSITIF"
    elif result['polarity'] < -0.1:
        sentiment = "NEGATIF"
    else:
        sentiment = "NEUTRE"
    
    print(f"\n\"{headline}\"")
    print(f"  Polarity: {result['polarity']:+.3f} ({sentiment})")
    print(f"  Subjectivity: {result['subjectivity']:.3f}")

### 2.2 VADER pour Textes Financiers (15 min)

**VADER** (Valence Aware Dictionary and sEntiment Reasoner) est specialement concu pour les textes courts et informels comme les tweets. Il gere mieux:

- **Emphase** : "AMAZING!!" vs "amazing"
- **Emoticons** : ":)" = positif
- **Modificateurs** : "very good" vs "good"
- **Negations** : "not good" = negatif
- **Argot** : "the bomb", "sick" (positifs)

VADER retourne un score `compound` entre -1 et +1 qui resume le sentiment global.

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Initialiser l'analyseur VADER
vader_analyzer = SentimentIntensityAnalyzer()

def analyze_sentiment_vader(text: str) -> Dict:
    """
    Analyse le sentiment d'un texte avec VADER.
    
    Args:
        text: Le texte a analyser
    
    Returns:
        Dict avec scores VADER (neg, neu, pos, compound)
    """
    scores = vader_analyzer.polarity_scores(text)
    return scores

def financial_sentiment(text: str) -> float:
    """
    Retourne le score compound VADER (-1 a +1).
    Optimise pour les textes financiers courts.
    """
    scores = vader_analyzer.polarity_scores(text)
    return scores['compound']

# Tester sur les memes headlines
print("Analyse de Sentiment avec VADER:")
print("="*70)

for headline in headlines:
    scores = analyze_sentiment_vader(headline)
    compound = scores['compound']
    
    # Interpretation standard VADER
    if compound >= 0.05:
        sentiment = "POSITIF"
    elif compound <= -0.05:
        sentiment = "NEGATIF"
    else:
        sentiment = "NEUTRE"
    
    print(f"\n\"{headline}\"")
    print(f"  Compound: {compound:+.3f} ({sentiment})")
    print(f"  Breakdown - Neg: {scores['neg']:.2f}, Neu: {scores['neu']:.2f}, Pos: {scores['pos']:.2f}")

In [ ]:
# Comparaison TextBlob vs VADER sur textes specifiques

test_texts = [
    "AMAZING earnings report!!! Stock is going to the MOON!!!",
    "This is a terrible disaster for shareholders :(",
    "Revenue was ok but not great",
    "$TSLA is absolutely crushing it today!!!",
    "I'm not impressed with the guidance at all",
    "The market is neither bullish nor bearish"
]

print("Comparaison TextBlob vs VADER:")
print("="*80)
print(f"{'Texte':<50} {'TextBlob':>10} {'VADER':>10}")
print("-"*80)

for text in test_texts:
    tb_score = analyze_sentiment_textblob(text)['polarity']
    vader_score = financial_sentiment(text)
    
    display_text = text[:47] + "..." if len(text) > 50 else text
    print(f"{display_text:<50} {tb_score:>+10.2f} {vader_score:>+10.2f}")

print("\nNote: VADER capture mieux l'emphase (CAPS, !!!) et les emoticons")

In [ ]:
# Visualisation de la distribution des scores

# Generer des headlines simules
np.random.seed(42)
simulated_headlines = [
    "Strong earnings beat expectations",
    "Stock price drops amid concerns",
    "Company announces new product launch",
    "Analyst upgrades rating to buy",
    "CEO resigns unexpectedly",
    "Quarterly revenue grows 25%",
    "Lawsuit filed against company",
    "Partnership announced with tech giant",
    "Layoffs planned for next quarter",
    "Dividend increased by 10%",
    "Stock hits all-time high",
    "Profit warning issued",
    "New market expansion planned",
    "Debt downgrade concerns investors",
    "Acquisition completed successfully",
    "Market share declines",
    "Innovation award received",
    "Regulatory investigation announced",
    "Strong guidance for next year",
    "Supply chain issues persist"
]

# Calculer les scores
tb_scores = [analyze_sentiment_textblob(h)['polarity'] for h in simulated_headlines]
vader_scores = [financial_sentiment(h) for h in simulated_headlines]

# Visualisation
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogrammes
ax1 = axes[0]
ax1.hist(tb_scores, bins=15, alpha=0.6, label='TextBlob', color='steelblue', edgecolor='black')
ax1.hist(vader_scores, bins=15, alpha=0.6, label='VADER', color='coral', edgecolor='black')
ax1.axvline(x=0, color='black', linestyle='--', linewidth=1)
ax1.set_xlabel('Score de Sentiment', fontsize=12)
ax1.set_ylabel('Frequence', fontsize=12)
ax1.set_title('Distribution des Scores de Sentiment', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Scatter plot comparaison
ax2 = axes[1]
ax2.scatter(tb_scores, vader_scores, alpha=0.7, s=80, c='navy')
ax2.plot([-1, 1], [-1, 1], 'r--', label='y=x', linewidth=1)
ax2.set_xlabel('TextBlob Score', fontsize=12)
ax2.set_ylabel('VADER Score', fontsize=12)
ax2.set_title('TextBlob vs VADER', fontsize=14, fontweight='bold')
ax2.set_xlim(-1.1, 1.1)
ax2.set_ylim(-1.1, 1.1)
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Correlation
correlation = np.corrcoef(tb_scores, vader_scores)[0, 1]
print(f"\nCorrelation TextBlob/VADER: {correlation:.2f}")

---

## Partie 3 : Integration dans QuantConnect (30 min)

### 3.1 Custom Sentiment Data Provider (15 min)

Pour utiliser des donnees de sentiment dans QuantConnect, nous devons creer un **Custom Data** provider. Cela permet d'integrer n'importe quelle source de donnees externes.

### Structure d'un Custom Data

```python
class SentimentData(PythonData):
    def GetSource(self, config, date, isLiveMode):
        # Retourne l'URL ou le chemin des donnees
        pass
    
    def Reader(self, config, line, date, isLiveMode):
        # Parse une ligne de donnees et retourne un objet
        pass
```

In [ ]:
# Custom Sentiment Data pour QuantConnect
# A copier dans l'IDE QuantConnect

sentiment_data_code = '''
from AlgorithmImports import *
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import json

class SentimentData(PythonData):
    """
    Custom Data pour integrer le sentiment de news.
    
    Cette classe permet de:
    1. Charger des donnees de news depuis une API ou fichier
    2. Analyser le sentiment avec VADER
    3. Rendre les scores disponibles dans l'algorithme
    """
    
    def __init__(self):
        super().__init__()
        # Initialiser VADER une fois
        self.analyzer = SentimentIntensityAnalyzer()
    
    def GetSource(self, config, date, isLiveMode):
        """
        Retourne la source des donnees.
        
        En mode backtest: fichier CSV local ou Dropbox
        En mode live: API NewsAPI ou autre
        """
        if isLiveMode:
            # URL de l\'API en mode live (exemple NewsAPI)
            # Remplacer API_KEY par votre cle
            ticker = config.Symbol.Value
            source = f"https://newsapi.org/v2/everything?q={ticker}&apiKey=YOUR_API_KEY"
        else:
            # En backtest, utiliser des donnees historiques (fichier)
            source = "https://www.dropbox.com/s/your-sentiment-data.csv?dl=1"
        
        return SubscriptionDataSource(
            source,
            SubscriptionTransportMedium.RemoteFile,
            FileFormat.UnfoldingCollection
        )
    
    def Reader(self, config, line, date, isLiveMode):
        """
        Parse une ligne de donnees et calcule le sentiment.
        
        Format attendu (JSON):
        {"published": "2024-01-15T10:30:00Z", "title": "...", "description": "..."}
        """
        if not line or line.startswith("#"):
            return None
        
        try:
            data = json.loads(line)
        except:
            return None
        
        # Creer l\'objet SentimentData
        sentiment = SentimentData()
        sentiment.Symbol = config.Symbol
        
        # Parser la date
        try:
            sentiment.Time = datetime.strptime(
                data.get("published", "")[:19], 
                "%Y-%m-%dT%H:%M:%S"
            )
        except:
            sentiment.Time = date
        
        # Concatener titre et description pour l\'analyse
        text = data.get("title", "") + " " + data.get("description", "")
        
        # Analyser le sentiment
        scores = self.analyzer.polarity_scores(text)
        
        # Stocker les scores
        sentiment["Compound"] = scores["compound"]
        sentiment["Positive"] = scores["pos"]
        sentiment["Negative"] = scores["neg"]
        sentiment["Neutral"] = scores["neu"]
        sentiment["Title"] = data.get("title", "")
        
        # Value = Compound (score principal)
        sentiment.Value = scores["compound"]
        
        return sentiment
'''

print("SentimentData Custom Data Provider:")
print(sentiment_data_code)

In [ ]:
# Utilisation du Custom Data dans un algorithme
# A copier dans l'IDE QuantConnect

usage_code = '''
from AlgorithmImports import *

class SentimentTradingAlgorithm(QCAlgorithm):
    """
    Algorithme qui trade sur le sentiment de news.
    """
    
    def Initialize(self):
        self.SetStartDate(2023, 1, 1)
        self.SetEndDate(2024, 1, 1)
        self.SetCash(100000)
        
        # Ajouter l\'equity
        self.symbol = self.AddEquity("AAPL", Resolution.Daily).Symbol
        
        # Ajouter les donnees de sentiment comme Custom Data
        self.sentiment_symbol = self.AddData(
            SentimentData,                    # Type de donnees
            "AAPL_SENTIMENT",                 # Ticker personnalise
            Resolution.Hour                   # Resolution
        ).Symbol
        
        # Historique du sentiment pour calcul de moyenne mobile
        self.sentiment_history = []
        self.sentiment_window = 7  # 7 jours de moyenne
    
    def OnData(self, data):
        # Verifier si on a des donnees de sentiment
        if self.sentiment_symbol in data:
            sentiment_data = data[self.sentiment_symbol]
            compound = sentiment_data.Value  # Score compound VADER
            
            # Ajouter a l\'historique
            self.sentiment_history.append(compound)
            
            # Garder seulement la fenetre
            if len(self.sentiment_history) > self.sentiment_window * 24:
                self.sentiment_history = self.sentiment_history[-self.sentiment_window * 24:]
            
            # Log le sentiment
            self.Debug(f"{self.Time}: Sentiment = {compound:.3f}")
        
        # Trading logic basee sur sentiment moyen
        if len(self.sentiment_history) >= self.sentiment_window:
            avg_sentiment = np.mean(self.sentiment_history)
            
            if avg_sentiment > 0.2 and not self.Portfolio[self.symbol].Invested:
                self.SetHoldings(self.symbol, 1.0)
                self.Log(f"BUY: Average sentiment = {avg_sentiment:.3f}")
            
            elif avg_sentiment < -0.2 and self.Portfolio[self.symbol].Invested:
                self.Liquidate(self.symbol)
                self.Log(f"SELL: Average sentiment = {avg_sentiment:.3f}")
'''

print("Exemple d'utilisation du SentimentData:")
print(usage_code)

### 3.2 Sentiment Alpha Model (15 min)

Creeons un **Alpha Model** qui utilise l'agregation du sentiment pour generer des Insights. C'est une approche plus modulaire compatible avec l'Algorithm Framework.

In [ ]:
# Sentiment Alpha Model pour QuantConnect
# A copier dans l'IDE QuantConnect

sentiment_alpha_code = '''
from AlgorithmImports import *
from datetime import timedelta
from collections import deque
import numpy as np

class SentimentAlphaModel(AlphaModel):
    """
    Alpha Model base sur l\'agregation du sentiment de news.
    
    Genere des Insights bases sur:
    - La moyenne mobile du sentiment
    - Les seuils de sentiment bullish/bearish
    """
    
    def __init__(self, 
                 lookback: int = 7,
                 bullish_threshold: float = 0.2,
                 bearish_threshold: float = -0.2,
                 insight_period: timedelta = timedelta(days=5)):
        """
        Initialise le Sentiment Alpha Model.
        
        Args:
            lookback: Nombre de jours pour la moyenne mobile
            bullish_threshold: Seuil pour signal bullish
            bearish_threshold: Seuil pour signal bearish
            insight_period: Duree de validite des Insights
        """
        self.lookback = lookback
        self.bullish_threshold = bullish_threshold
        self.bearish_threshold = bearish_threshold
        self.insight_period = insight_period
        
        # Historique du sentiment par symbole
        self.sentiment_history = {}
        
        # Mapping entre symbole sentiment et symbole equity
        self.sentiment_to_equity = {}
    
    def Update(self, algorithm, data):
        """
        Genere des Insights bases sur le sentiment agrege.
        
        Args:
            algorithm: Reference a l\'algorithme
            data: Donnees de marche
        
        Returns:
            Liste d\'Insights
        """
        insights = []
        
        # Mettre a jour l\'historique avec les nouvelles donnees de sentiment
        for symbol in self.sentiment_history.keys():
            sentiment_symbol = self._get_sentiment_symbol(symbol)
            
            if sentiment_symbol in data:
                sentiment_value = data[sentiment_symbol].Value
                self.sentiment_history[symbol].append(sentiment_value)
                algorithm.Debug(f"{symbol.Value}: New sentiment = {sentiment_value:.3f}")
        
        # Generer des Insights pour chaque symbole
        for symbol, history in self.sentiment_history.items():
            if len(history) < self.lookback:
                continue
            
            # Calculer la moyenne mobile du sentiment
            recent_sentiment = list(history)[-self.lookback:]
            avg_sentiment = np.mean(recent_sentiment)
            
            # Calculer la tendance (sentiment actuel vs moyenne)
            current_sentiment = history[-1] if len(history) > 0 else 0
            sentiment_momentum = current_sentiment - avg_sentiment
            
            # Generer les Insights
            if avg_sentiment > self.bullish_threshold:
                # Signal haussier
                confidence = min(abs(avg_sentiment) / 0.5, 1.0)  # Scale to 0-1
                insights.append(Insight.Price(
                    symbol,
                    self.insight_period,
                    InsightDirection.Up,
                    magnitude=abs(avg_sentiment),
                    confidence=confidence * 0.6  # Max 60% confidence
                ))
                algorithm.Debug(f"UP Insight: {symbol.Value}, Sentiment={avg_sentiment:.3f}")
            
            elif avg_sentiment < self.bearish_threshold:
                # Signal baissier
                confidence = min(abs(avg_sentiment) / 0.5, 1.0)
                insights.append(Insight.Price(
                    symbol,
                    self.insight_period,
                    InsightDirection.Down,
                    magnitude=abs(avg_sentiment),
                    confidence=confidence * 0.6
                ))
                algorithm.Debug(f"DOWN Insight: {symbol.Value}, Sentiment={avg_sentiment:.3f}")
        
        return insights
    
    def OnSecuritiesChanged(self, algorithm, changes):
        """
        Gere les ajouts/suppressions de securities.
        Initialise l\'historique pour les nouvelles securities.
        """
        for security in changes.AddedSecurities:
            symbol = security.Symbol
            if symbol not in self.sentiment_history:
                # Creer un deque avec une taille max
                self.sentiment_history[symbol] = deque(maxlen=self.lookback * 5)
        
        for security in changes.RemovedSecurities:
            symbol = security.Symbol
            if symbol in self.sentiment_history:
                del self.sentiment_history[symbol]
    
    def _get_sentiment_symbol(self, equity_symbol):
        """
        Retourne le symbole de donnees de sentiment correspondant.
        """
        # Convention: TICKER_SENTIMENT
        return Symbol.Create(
            f"{equity_symbol.Value}_SENTIMENT",
            SecurityType.Base,
            Market.USA
        )
'''

print("SentimentAlphaModel:")
print(sentiment_alpha_code)

In [ ]:
# Simulation locale du SentimentAlphaModel
from collections import deque

class SentimentAlphaModelSimulator:
    """
    Version simplifiee du SentimentAlphaModel pour simulation locale.
    """
    
    def __init__(self, lookback=7, bullish_threshold=0.2, bearish_threshold=-0.2):
        self.lookback = lookback
        self.bullish_threshold = bullish_threshold
        self.bearish_threshold = bearish_threshold
        self.sentiment_history = deque(maxlen=lookback * 5)
    
    def add_sentiment(self, value):
        """Ajoute une valeur de sentiment."""
        self.sentiment_history.append(value)
    
    def get_signal(self):
        """Retourne le signal actuel."""
        if len(self.sentiment_history) < self.lookback:
            return 'WAIT', 0, 0
        
        avg = np.mean(list(self.sentiment_history)[-self.lookback:])
        confidence = min(abs(avg) / 0.5, 1.0) * 0.6
        
        if avg > self.bullish_threshold:
            return 'BUY', avg, confidence
        elif avg < self.bearish_threshold:
            return 'SELL', avg, confidence
        else:
            return 'HOLD', avg, 0

# Simuler avec des donnees
np.random.seed(42)

# Generer une serie de sentiment avec tendances
n_days = 60
dates = pd.date_range('2024-01-01', periods=n_days, freq='D')

# Sentiment avec tendance: positif puis negatif puis positif
trend = np.concatenate([
    np.linspace(0, 0.4, 20),
    np.linspace(0.4, -0.3, 20),
    np.linspace(-0.3, 0.2, 20)
])
noise = np.random.randn(n_days) * 0.1
sentiment_values = np.clip(trend + noise, -1, 1)

# Simuler le trading
alpha = SentimentAlphaModelSimulator(lookback=7)

signals = []
avg_sentiments = []

for i, sent in enumerate(sentiment_values):
    alpha.add_sentiment(sent)
    signal, avg, conf = alpha.get_signal()
    signals.append(signal)
    avg_sentiments.append(avg)

# Visualisation
fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

# Sentiment brut et moyenne
ax1 = axes[0]
ax1.plot(dates, sentiment_values, 'b-', alpha=0.5, label='Sentiment Brut')
ax1.plot(dates, avg_sentiments, 'r-', linewidth=2, label=f'Moyenne Mobile ({alpha.lookback}j)')
ax1.axhline(y=0.2, color='green', linestyle='--', alpha=0.5, label='Seuil Bullish')
ax1.axhline(y=-0.2, color='red', linestyle='--', alpha=0.5, label='Seuil Bearish')
ax1.axhline(y=0, color='black', linestyle='-', alpha=0.3)
ax1.fill_between(dates, 0.2, 1, alpha=0.1, color='green')
ax1.fill_between(dates, -1, -0.2, alpha=0.1, color='red')
ax1.set_ylabel('Sentiment', fontsize=12)
ax1.set_title('Simulation du Sentiment Alpha Model', fontsize=14, fontweight='bold')
ax1.legend(loc='upper right')
ax1.grid(True, alpha=0.3)

# Signaux
ax2 = axes[1]
colors = {'BUY': 'green', 'SELL': 'red', 'HOLD': 'gray', 'WAIT': 'lightgray'}
for i, (date, signal) in enumerate(zip(dates, signals)):
    ax2.bar(date, 1, color=colors[signal], edgecolor='black', linewidth=0.5)
ax2.set_ylabel('Signal', fontsize=12)
ax2.set_xlabel('Date', fontsize=12)
ax2.set_yticks([])
ax2.set_title('Signaux de Trading', fontsize=14, fontweight='bold')

# Legende custom
from matplotlib.patches import Patch
legend_elements = [Patch(facecolor=c, label=l) for l, c in colors.items()]
ax2.legend(handles=legend_elements, loc='upper right')

plt.tight_layout()
plt.show()

# Stats
signal_counts = pd.Series(signals).value_counts()
print("\nDistribution des signaux:")
for sig, count in signal_counts.items():
    print(f"  {sig}: {count} ({count/len(signals)*100:.1f}%)")

---

## Partie 4 : Sentiment Contrarien (15 min)

### 4.1 Theorie du Sentiment Contrarien

Le trading contrarien suppose que les **extremes de sentiment** sont des signaux de retournement:

| Sentiment | Interpretation | Action Contrarienne |
|-----------|---------------|--------------------|
| **Tres Positif (> 0.7)** | Euphorie, "tout le monde achete" | **VENDRE** (potentiel sommet) |
| **Tres Negatif (< -0.7)** | Panique, "tout le monde vend" | **ACHETER** (potentiel creux) |
| **Modere** | Pas d'extreme | Pas de signal contrarien |

### Indicateurs Contrariens Historiques

- **VIX eleve** : Peur extreme -> opportunite d'achat
- **Put/Call ratio eleve** : Pessimisme extreme -> acheter
- **Magazine covers** : Quand les medias mainstream sont unanimement bullish -> vendre

In [ ]:
# Strategie Contrarienne base sur le sentiment
# A copier dans l'IDE QuantConnect

contrarian_alpha_code = '''
from AlgorithmImports import *
from datetime import timedelta
from collections import deque
import numpy as np

class ContrarianSentimentAlphaModel(AlphaModel):
    """
    Alpha Model contrarien: trade contre les extremes de sentiment.
    
    Logique:
    - Sentiment > euphoria_threshold -> Insight DOWN (vendre)
    - Sentiment < panic_threshold -> Insight UP (acheter)
    """
    
    def __init__(self,
                 lookback: int = 7,
                 euphoria_threshold: float = 0.7,
                 panic_threshold: float = -0.7,
                 insight_period: timedelta = timedelta(days=3)):
        """
        Initialise le modele contrarien.
        
        Args:
            lookback: Fenetre pour la moyenne mobile
            euphoria_threshold: Seuil de sentiment extreme positif
            panic_threshold: Seuil de sentiment extreme negatif
            insight_period: Duree des insights (court pour contrarien)
        """
        self.lookback = lookback
        self.euphoria_threshold = euphoria_threshold
        self.panic_threshold = panic_threshold
        self.insight_period = insight_period
        self.sentiment_history = {}
    
    def Update(self, algorithm, data):
        insights = []
        
        for symbol, history in self.sentiment_history.items():
            if len(history) < self.lookback:
                continue
            
            # Calculer le sentiment moyen
            avg_sentiment = np.mean(list(history)[-self.lookback:])
            
            # === LOGIQUE CONTRARIENNE ===
            if avg_sentiment > self.euphoria_threshold:
                # Euphorie detectee -> Contrarien: VENDRE
                confidence = min((avg_sentiment - self.euphoria_threshold) / 0.3, 1.0)
                insights.append(Insight.Price(
                    symbol,
                    self.insight_period,
                    InsightDirection.Down,  # Contrarien!
                    magnitude=abs(avg_sentiment),
                    confidence=confidence * 0.5
                ))
                algorithm.Log(f"CONTRARIAN SELL: {symbol.Value}, Euphoria={avg_sentiment:.3f}")
            
            elif avg_sentiment < self.panic_threshold:
                # Panique detectee -> Contrarien: ACHETER
                confidence = min((self.panic_threshold - avg_sentiment) / 0.3, 1.0)
                insights.append(Insight.Price(
                    symbol,
                    self.insight_period,
                    InsightDirection.Up,  # Contrarien!
                    magnitude=abs(avg_sentiment),
                    confidence=confidence * 0.5
                ))
                algorithm.Log(f"CONTRARIAN BUY: {symbol.Value}, Panic={avg_sentiment:.3f}")
        
        return insights
    
    def OnSecuritiesChanged(self, algorithm, changes):
        for security in changes.AddedSecurities:
            symbol = security.Symbol
            if symbol not in self.sentiment_history:
                self.sentiment_history[symbol] = deque(maxlen=self.lookback * 3)
        
        for security in changes.RemovedSecurities:
            symbol = security.Symbol
            if symbol in self.sentiment_history:
                del self.sentiment_history[symbol]
'''

print("ContrarianSentimentAlphaModel:")
print(contrarian_alpha_code)

In [ ]:
# Simulation de la strategie contrarienne

class ContrarianSimulator:
    """Simulation locale de la strategie contrarienne."""
    
    def __init__(self, lookback=7, euphoria=0.6, panic=-0.6):
        self.lookback = lookback
        self.euphoria = euphoria
        self.panic = panic
        self.history = deque(maxlen=lookback * 3)
    
    def add_sentiment(self, value):
        self.history.append(value)
    
    def get_signal(self):
        if len(self.history) < self.lookback:
            return 'WAIT', 0
        
        avg = np.mean(list(self.history)[-self.lookback:])
        
        if avg > self.euphoria:
            return 'CONTRARIAN_SELL', avg  # Vendre quand tout le monde est euphoric
        elif avg < self.panic:
            return 'CONTRARIAN_BUY', avg   # Acheter quand tout le monde panique
        return 'HOLD', avg

# Generer des prix et sentiment correles
np.random.seed(123)
n = 100
dates = pd.date_range('2024-01-01', periods=n, freq='D')

# Prix avec bulle puis crash puis recovery
price_base = 100
prices = [price_base]
sentiment_series = []

for i in range(1, n):
    if i < 30:
        # Phase haussiere
        ret = 0.01 + np.random.randn() * 0.01
        sent = 0.3 + i * 0.015 + np.random.randn() * 0.1  # Sentiment monte
    elif i < 50:
        # Sommet / Euphorie
        ret = 0.005 + np.random.randn() * 0.015
        sent = 0.7 + np.random.randn() * 0.1  # Euphorie
    elif i < 70:
        # Crash
        ret = -0.02 + np.random.randn() * 0.02
        sent = -0.3 - (i - 50) * 0.02 + np.random.randn() * 0.1  # Panique
    else:
        # Recovery
        ret = 0.01 + np.random.randn() * 0.01
        sent = -0.5 + (i - 70) * 0.02 + np.random.randn() * 0.1
    
    prices.append(prices[-1] * (1 + ret))
    sentiment_series.append(np.clip(sent, -1, 1))

prices = np.array(prices[1:])
sentiment_series = np.array(sentiment_series)

# Simuler le trading contrarien
contrarian = ContrarianSimulator(lookback=5, euphoria=0.6, panic=-0.6)

signals = []
for sent in sentiment_series:
    contrarian.add_sentiment(sent)
    sig, _ = contrarian.get_signal()
    signals.append(sig)

# Visualisation
fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)

# Prix
ax1 = axes[0]
ax1.plot(dates, prices, 'b-', linewidth=2)
# Marquer les signaux
for i, (date, sig) in enumerate(zip(dates, signals)):
    if sig == 'CONTRARIAN_BUY':
        ax1.axvline(x=date, color='green', alpha=0.3)
        ax1.scatter([date], [prices[i]], color='green', s=100, marker='^', zorder=5)
    elif sig == 'CONTRARIAN_SELL':
        ax1.axvline(x=date, color='red', alpha=0.3)
        ax1.scatter([date], [prices[i]], color='red', s=100, marker='v', zorder=5)
ax1.set_ylabel('Prix', fontsize=12)
ax1.set_title('Strategie Contrarienne sur le Sentiment', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)

# Sentiment
ax2 = axes[1]
ax2.plot(dates, sentiment_series, 'purple', linewidth=1.5)
ax2.axhline(y=0.6, color='red', linestyle='--', label='Seuil Euphorie')
ax2.axhline(y=-0.6, color='green', linestyle='--', label='Seuil Panique')
ax2.axhline(y=0, color='black', linestyle='-', alpha=0.3)
ax2.fill_between(dates, 0.6, 1, alpha=0.2, color='red', label='Zone Euphorie')
ax2.fill_between(dates, -1, -0.6, alpha=0.2, color='green', label='Zone Panique')
ax2.set_ylabel('Sentiment', fontsize=12)
ax2.legend(loc='upper right')
ax2.grid(True, alpha=0.3)

# Signaux
ax3 = axes[2]
colors_map = {'CONTRARIAN_BUY': 'green', 'CONTRARIAN_SELL': 'red', 'HOLD': 'gray', 'WAIT': 'lightgray'}
for i, (date, sig) in enumerate(zip(dates, signals)):
    ax3.bar(date, 1, color=colors_map[sig], edgecolor='none')
ax3.set_ylabel('Signal', fontsize=12)
ax3.set_xlabel('Date', fontsize=12)
ax3.set_yticks([])
legend_elements = [Patch(facecolor=c, label=l.replace('CONTRARIAN_', '')) 
                   for l, c in colors_map.items() if 'CONTRARIAN' in l or l == 'HOLD']
ax3.legend(handles=legend_elements, loc='upper right')

plt.tight_layout()
plt.show()

# Analyse des signaux
buy_signals = [i for i, s in enumerate(signals) if s == 'CONTRARIAN_BUY']
sell_signals = [i for i, s in enumerate(signals) if s == 'CONTRARIAN_SELL']

print(f"\nAnalyse des signaux contrariens:")
print(f"  Signaux BUY: {len(buy_signals)} (en zone de panique)")
print(f"  Signaux SELL: {len(sell_signals)} (en zone d'euphorie)")

# Performance hypothetique
if buy_signals and sell_signals:
    avg_buy_price = np.mean([prices[i] for i in buy_signals])
    avg_sell_price = np.mean([prices[i] for i in sell_signals])
    print(f"\n  Prix moyen d'achat (panique): ${avg_buy_price:.2f}")
    print(f"  Prix moyen de vente (euphorie): ${avg_sell_price:.2f}")
    print(f"  Gain theorique: {(avg_sell_price/avg_buy_price - 1)*100:.1f}%")

---

## Partie 5 : Combinaison Sentiment + Technique (20 min)

### 5.1 Pourquoi Combiner?

Le sentiment seul peut etre bruyant et peu fiable. La combinaison avec l'analyse technique permet de:

| Benefice | Description |
|----------|-------------|
| **Confirmation** | Le sentiment confirme le signal technique |
| **Filtre** | Eviter les faux signaux |
| **Timing** | Le technique donne le timing, le sentiment la direction |
| **Robustesse** | Diversification des sources de signaux |

### Regles de Combinaison

```
Signal d'ACHAT = Sentiment Positif (> 0.1)
                 AND Prix > SMA 20 (tendance haussiere)
                 AND RSI < 70 (pas suracheté)

Signal de VENTE = Sentiment Negatif (< -0.1)
                  AND Prix < SMA 20 (tendance baissiere)
                  AND RSI > 30 (pas survendu)
```

In [ ]:
# Hybrid Alpha Model: Sentiment + Technique
# A copier dans l'IDE QuantConnect

hybrid_alpha_code = '''
from AlgorithmImports import *
from datetime import timedelta
from collections import deque
import numpy as np

class HybridSentimentTechnicalAlpha(AlphaModel):
    """
    Alpha Model hybride combinant:
    - Sentiment de news (direction)
    - Indicateurs techniques (confirmation)
    
    Logique:
    - Signal BUY: Sentiment > threshold AND Prix > SMA AND RSI < 70
    - Signal SELL: Sentiment < -threshold AND Prix < SMA AND RSI > 30
    """
    
    def __init__(self,
                 sentiment_lookback: int = 5,
                 sentiment_threshold: float = 0.1,
                 sma_period: int = 20,
                 rsi_period: int = 14):
        """
        Args:
            sentiment_lookback: Fenetre pour moyenne sentiment
            sentiment_threshold: Seuil sentiment pour signal
            sma_period: Periode de la SMA pour tendance
            rsi_period: Periode du RSI
        """
        self.sentiment_lookback = sentiment_lookback
        self.sentiment_threshold = sentiment_threshold
        self.sma_period = sma_period
        self.rsi_period = rsi_period
        
        self.symbolData = {}
    
    def Update(self, algorithm, data):
        insights = []
        
        for symbol, sd in self.symbolData.items():
            if not sd.IsReady:
                continue
            
            if not data.ContainsKey(symbol):
                continue
            
            price = data[symbol].Close
            
            # Recuperer les valeurs
            avg_sentiment = sd.AverageSentiment
            sma = sd.SMA.Current.Value
            rsi = sd.RSI.Current.Value
            
            # === LOGIQUE HYBRIDE ===
            # Signal d\'achat
            if (avg_sentiment > self.sentiment_threshold and
                price > sma and
                rsi < 70):
                
                # Calculer la confiance basee sur la force des signaux
                sentiment_strength = min(avg_sentiment / 0.3, 1.0)
                trend_strength = min((price - sma) / sma / 0.02, 1.0)
                rsi_room = (70 - rsi) / 40  # Plus de marge = plus de confiance
                
                confidence = (sentiment_strength + trend_strength + rsi_room) / 3
                confidence = min(confidence * 0.7, 0.7)
                
                insights.append(Insight.Price(
                    symbol,
                    timedelta(days=5),
                    InsightDirection.Up,
                    magnitude=avg_sentiment,
                    confidence=confidence
                ))
                algorithm.Debug(f"HYBRID BUY: {symbol.Value}, Sent={avg_sentiment:.2f}, SMA={price>sma}, RSI={rsi:.0f}")
            
            # Signal de vente
            elif (avg_sentiment < -self.sentiment_threshold and
                  price < sma and
                  rsi > 30):
                
                sentiment_strength = min(abs(avg_sentiment) / 0.3, 1.0)
                trend_strength = min((sma - price) / sma / 0.02, 1.0)
                rsi_room = (rsi - 30) / 40
                
                confidence = (sentiment_strength + trend_strength + rsi_room) / 3
                confidence = min(confidence * 0.7, 0.7)
                
                insights.append(Insight.Price(
                    symbol,
                    timedelta(days=5),
                    InsightDirection.Down,
                    magnitude=abs(avg_sentiment),
                    confidence=confidence
                ))
                algorithm.Debug(f"HYBRID SELL: {symbol.Value}, Sent={avg_sentiment:.2f}, SMA={price<sma}, RSI={rsi:.0f}")
        
        return insights
    
    def OnSecuritiesChanged(self, algorithm, changes):
        for security in changes.AddedSecurities:
            symbol = security.Symbol
            if symbol not in self.symbolData:
                self.symbolData[symbol] = HybridSymbolData(
                    algorithm, symbol,
                    self.sentiment_lookback,
                    self.sma_period,
                    self.rsi_period
                )
        
        for security in changes.RemovedSecurities:
            if security.Symbol in self.symbolData:
                del self.symbolData[security.Symbol]


class HybridSymbolData:
    """
    Donnees par symbole pour le modele hybride.
    Contient: historique sentiment + indicateurs techniques.
    """
    
    def __init__(self, algorithm, symbol, sentiment_lookback, sma_period, rsi_period):
        self.Symbol = symbol
        self.sentiment_lookback = sentiment_lookback
        
        # Historique sentiment
        self.sentiment_history = deque(maxlen=sentiment_lookback * 2)
        
        # Indicateurs techniques
        self.SMA = algorithm.SMA(symbol, sma_period, Resolution.Daily)
        self.RSI = algorithm.RSI(symbol, rsi_period, Resolution.Daily)
        
        # Warmup
        history = algorithm.History(symbol, max(sma_period, rsi_period), Resolution.Daily)
        if not history.empty:
            for bar in history.itertuples():
                self.SMA.Update(bar.Index[1], bar.close)
                self.RSI.Update(bar.Index[1], bar.close)
    
    def AddSentiment(self, value):
        """Ajoute une valeur de sentiment."""
        self.sentiment_history.append(value)
    
    @property
    def AverageSentiment(self):
        """Retourne la moyenne du sentiment."""
        if len(self.sentiment_history) < self.sentiment_lookback:
            return 0
        return np.mean(list(self.sentiment_history)[-self.sentiment_lookback:])
    
    @property
    def IsReady(self):
        """Tous les indicateurs sont prets."""
        return (self.SMA.IsReady and 
                self.RSI.IsReady and 
                len(self.sentiment_history) >= self.sentiment_lookback)
'''

print("HybridSentimentTechnicalAlpha:")
print(hybrid_alpha_code)

In [ ]:
# Simulation locale de la strategie hybride

def calculate_sma(prices, period):
    """Calcule la SMA."""
    return pd.Series(prices).rolling(window=period).mean().values

def calculate_rsi(prices, period=14):
    """Calcule le RSI."""
    prices = pd.Series(prices)
    delta = prices.diff()
    gain = delta.where(delta > 0, 0).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi.values

# Generer des donnees
np.random.seed(456)
n = 120
dates = pd.date_range('2024-01-01', periods=n, freq='D')

# Prix avec tendance et cycles
trend = np.linspace(100, 130, n)
cycle = 10 * np.sin(np.linspace(0, 4 * np.pi, n))
noise = np.cumsum(np.random.randn(n) * 0.3)
prices = trend + cycle + noise

# Sentiment correle au cycle (avec decalage et bruit)
sentiment = 0.4 * np.sin(np.linspace(0.5, 4.5 * np.pi, n)) + np.random.randn(n) * 0.15
sentiment = np.clip(sentiment, -1, 1)

# Calculer indicateurs
sma = calculate_sma(prices, 20)
rsi = calculate_rsi(prices, 14)

# Calculer sentiment moyen mobile
sentiment_ma = pd.Series(sentiment).rolling(window=5).mean().values

# Generer signaux hybrides
hybrid_signals = []
threshold = 0.1

for i in range(len(prices)):
    if np.isnan(sma[i]) or np.isnan(rsi[i]) or np.isnan(sentiment_ma[i]):
        hybrid_signals.append('WAIT')
        continue
    
    sent = sentiment_ma[i]
    price = prices[i]
    sma_val = sma[i]
    rsi_val = rsi[i]
    
    # Signal d'achat
    if sent > threshold and price > sma_val and rsi_val < 70:
        hybrid_signals.append('BUY')
    # Signal de vente
    elif sent < -threshold and price < sma_val and rsi_val > 30:
        hybrid_signals.append('SELL')
    else:
        hybrid_signals.append('HOLD')

# Visualisation
fig, axes = plt.subplots(4, 1, figsize=(14, 12), sharex=True)

# Prix et SMA
ax1 = axes[0]
ax1.plot(dates, prices, 'b-', label='Prix', linewidth=1.5)
ax1.plot(dates, sma, 'orange', label='SMA(20)', linewidth=1.5)
# Marquer les signaux
for i, sig in enumerate(hybrid_signals):
    if sig == 'BUY':
        ax1.scatter([dates[i]], [prices[i]], color='green', s=50, marker='^', zorder=5)
    elif sig == 'SELL':
        ax1.scatter([dates[i]], [prices[i]], color='red', s=50, marker='v', zorder=5)
ax1.set_ylabel('Prix', fontsize=12)
ax1.set_title('Strategie Hybride: Sentiment + Technique', fontsize=14, fontweight='bold')
ax1.legend(loc='upper left')
ax1.grid(True, alpha=0.3)

# RSI
ax2 = axes[1]
ax2.plot(dates, rsi, 'purple', linewidth=1.5)
ax2.axhline(y=70, color='red', linestyle='--', alpha=0.5)
ax2.axhline(y=30, color='green', linestyle='--', alpha=0.5)
ax2.fill_between(dates, 70, 100, alpha=0.1, color='red')
ax2.fill_between(dates, 0, 30, alpha=0.1, color='green')
ax2.set_ylabel('RSI', fontsize=12)
ax2.set_ylim(0, 100)
ax2.grid(True, alpha=0.3)

# Sentiment
ax3 = axes[2]
ax3.plot(dates, sentiment, 'gray', alpha=0.5, label='Sentiment brut')
ax3.plot(dates, sentiment_ma, 'blue', linewidth=2, label='Sentiment MA(5)')
ax3.axhline(y=0.1, color='green', linestyle='--', alpha=0.5)
ax3.axhline(y=-0.1, color='red', linestyle='--', alpha=0.5)
ax3.axhline(y=0, color='black', linestyle='-', alpha=0.3)
ax3.set_ylabel('Sentiment', fontsize=12)
ax3.legend(loc='upper right')
ax3.grid(True, alpha=0.3)

# Signaux
ax4 = axes[3]
colors_map = {'BUY': 'green', 'SELL': 'red', 'HOLD': 'gray', 'WAIT': 'lightgray'}
for i, (date, sig) in enumerate(zip(dates, hybrid_signals)):
    ax4.bar(date, 1, color=colors_map[sig], edgecolor='none')
ax4.set_ylabel('Signal', fontsize=12)
ax4.set_xlabel('Date', fontsize=12)
ax4.set_yticks([])
legend_elements = [Patch(facecolor=c, label=l) for l, c in colors_map.items() if l != 'WAIT']
ax4.legend(handles=legend_elements, loc='upper right')

plt.tight_layout()
plt.show()

# Statistiques
signal_counts = pd.Series(hybrid_signals).value_counts()
print("\nStatistiques des signaux hybrides:")
for sig, count in signal_counts.items():
    print(f"  {sig}: {count} ({count/len(hybrid_signals)*100:.1f}%)")

print("\nConditions pour signal BUY:")
print("  - Sentiment MA > 0.1")
print("  - Prix > SMA(20)")
print("  - RSI < 70")

---

## Partie 6 : Strategie Complete (20 min)

### 6.1 News Sentiment Trading Strategy

Assemblons tous les composants pour creer une strategie complete de trading sur sentiment.

### Architecture de la Strategie

```
                    News API
                       |
                       v
               +--------------+
               | VADER        |
               | Sentiment    |
               | Analysis     |
               +--------------+
                       |
                       v
               +--------------+
               | Sentiment    |
               | Aggregation  |---> Moyenne Mobile 3-5 jours
               +--------------+
                       |
           +-----------+-----------+
           |                       |
           v                       v
   +---------------+       +---------------+
   | Trend Filter  |       | RSI Filter    |
   | (SMA 50)      |       | (Not extreme) |
   +---------------+       +---------------+
           |                       |
           +-----------+-----------+
                       |
                       v
               +--------------+
               | Portfolio    |
               | Construction |
               +--------------+
                       |
                       v
               +--------------+
               | Risk Mgmt    |
               | (5% Stop)    |
               +--------------+
```

In [ ]:
# Strategie complete de News Sentiment Trading
# A copier dans l'IDE QuantConnect

complete_strategy_code = '''
from AlgorithmImports import *
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from datetime import timedelta
from collections import deque
import numpy as np


class NewsSentimentStrategy(QCAlgorithm):
    """
    Strategie complete de trading sur sentiment de news.
    
    Composants:
    - Source: NewsAPI (simulation via Custom Data)
    - Analyse: VADER Sentiment
    - Signal: Moyenne mobile 3 jours
    - Filtre: SMA 50 + RSI
    - Position sizing: Par confiance
    """
    
    def Initialize(self):
        # === CONFIGURATION ===
        self.SetStartDate(2023, 1, 1)
        self.SetEndDate(2024, 1, 1)
        self.SetCash(100000)
        
        # Parametres de la strategie
        self.sentiment_lookback = 3          # Moyenne sur 3 jours
        self.sentiment_threshold = 0.15      # Seuil pour signal
        self.sma_period = 50                 # SMA pour filtre de tendance
        self.rsi_period = 14                 # RSI pour filtre
        self.max_position_size = 0.2         # 20% max par position
        self.stop_loss_pct = 0.05            # 5% stop loss
        
        # === UNIVERS ===
        # Liste de tickers a trader
        self.tickers = ["AAPL", "MSFT", "GOOGL", "AMZN", "META"]
        
        # Ajouter les equities
        self.symbols = {}
        for ticker in self.tickers:
            equity = self.AddEquity(ticker, Resolution.Daily)
            self.symbols[ticker] = equity.Symbol
        
        # === INDICATEURS TECHNIQUES ===
        self.sma = {}
        self.rsi = {}
        for ticker, symbol in self.symbols.items():
            self.sma[symbol] = self.SMA(symbol, self.sma_period, Resolution.Daily)
            self.rsi[symbol] = self.RSI(symbol, self.rsi_period, Resolution.Daily)
        
        # === SENTIMENT DATA ===
        self.sentiment_history = {symbol: deque(maxlen=self.sentiment_lookback * 2) 
                                   for symbol in self.symbols.values()}
        self.vader = SentimentIntensityAnalyzer()
        
        # === TRACKING ===
        self.entry_prices = {}
        
        # Warmup
        self.SetWarmup(self.sma_period)
        
        # Schedule: Analyser le sentiment chaque jour a 9h30
        self.Schedule.On(
            self.DateRules.EveryDay(),
            self.TimeRules.AfterMarketOpen("SPY", 30),
            self.AnalyzeSentiment
        )
        
        self.Log("NewsSentimentStrategy initialized")
        self.Log(f"  Tickers: {self.tickers}")
        self.Log(f"  Sentiment lookback: {self.sentiment_lookback} days")
        self.Log(f"  Threshold: {self.sentiment_threshold}")
    
    def AnalyzeSentiment(self):
        """
        Simule l\'analyse de sentiment quotidienne.
        En production: appellerait NewsAPI et analyserait les headlines.
        """
        for ticker, symbol in self.symbols.items():
            # En production: fetcher les news via API
            # headlines = self.FetchNewsHeadlines(ticker)
            # sentiment = self.AnalyzeHeadlines(headlines)
            
            # Simulation: sentiment base sur le momentum du prix
            if self.sma[symbol].IsReady:
                price = self.Securities[symbol].Price
                sma_val = self.sma[symbol].Current.Value
                
                # Simuler un sentiment correle a la tendance
                price_momentum = (price - sma_val) / sma_val
                noise = (np.random.random() - 0.5) * 0.3
                simulated_sentiment = np.clip(price_momentum * 2 + noise, -1, 1)
                
                self.sentiment_history[symbol].append(simulated_sentiment)
                self.Debug(f"{self.Time.date()}: {ticker} sentiment = {simulated_sentiment:.3f}")
    
    def OnData(self, data):
        if self.IsWarmingUp:
            return
        
        for ticker, symbol in self.symbols.items():
            if not data.ContainsKey(symbol):
                continue
            
            # Verifier indicateurs prets
            if not (self.sma[symbol].IsReady and self.rsi[symbol].IsReady):
                continue
            
            # Verifier historique sentiment
            if len(self.sentiment_history[symbol]) < self.sentiment_lookback:
                continue
            
            price = data[symbol].Close
            
            # === CHECK STOP LOSS ===
            if self.Portfolio[symbol].Invested and symbol in self.entry_prices:
                pnl_pct = (price - self.entry_prices[symbol]) / self.entry_prices[symbol]
                if pnl_pct <= -self.stop_loss_pct:
                    self.Liquidate(symbol)
                    self.Log(f"{self.Time.date()}: STOP LOSS {ticker} at {pnl_pct:.2%}")
                    del self.entry_prices[symbol]
                    continue
            
            # === CALCULER SIGNAL ===
            avg_sentiment = np.mean(list(self.sentiment_history[symbol])[-self.sentiment_lookback:])
            sma_val = self.sma[symbol].Current.Value
            rsi_val = self.rsi[symbol].Current.Value
            
            # === LOGIQUE DE TRADING ===
            if not self.Portfolio[symbol].Invested:
                # Signal d\'achat
                if (avg_sentiment > self.sentiment_threshold and
                    price > sma_val and
                    rsi_val < 70):
                    
                    # Position sizing par confiance
                    confidence = min(avg_sentiment / 0.4, 1.0)
                    position_size = self.max_position_size * confidence
                    
                    self.SetHoldings(symbol, position_size)
                    self.entry_prices[symbol] = price
                    self.Log(f"{self.Time.date()}: BUY {ticker}, Sent={avg_sentiment:.2f}, Size={position_size:.1%}")
            
            else:
                # Signal de vente
                if (avg_sentiment < -self.sentiment_threshold or
                    price < sma_val or
                    rsi_val > 80):
                    
                    self.Liquidate(symbol)
                    if symbol in self.entry_prices:
                        pnl = (price - self.entry_prices[symbol]) / self.entry_prices[symbol]
                        self.Log(f"{self.Time.date()}: SELL {ticker}, PnL={pnl:.2%}")
                        del self.entry_prices[symbol]
    
    def OnEndOfAlgorithm(self):
        """Resume final."""
        self.Log("="*60)
        self.Log("NEWS SENTIMENT STRATEGY - FINAL SUMMARY")
        self.Log("="*60)
        self.Log(f"Final Portfolio Value: ${self.Portfolio.TotalPortfolioValue:,.2f}")
        total_return = (self.Portfolio.TotalPortfolioValue - 100000) / 100000
        self.Log(f"Total Return: {total_return:.2%}")
        self.Log("="*60)
'''

print("NewsSentimentStrategy Complete:")
print(complete_strategy_code)

In [ ]:
# Resume des composants de la strategie

print("="*70)
print("RESUME: News Sentiment Trading Strategy")
print("="*70)

components = {
    'Source de Donnees': {
        'Type': 'NewsAPI (gratuit/payant)',
        'Frequence': 'Quotidienne',
        'Contenu': 'Headlines + descriptions'
    },
    'Analyse de Sentiment': {
        'Outil': 'VADER Sentiment Analyzer',
        'Score': 'Compound (-1 a +1)',
        'Agregation': 'Moyenne mobile 3 jours'
    },
    'Filtres Techniques': {
        'Tendance': 'SMA 50 (prix > SMA = bullish)',
        'Momentum': 'RSI 14 (< 70 pour achat, > 30 pour vente)',
        'Confirmation': 'Les 3 conditions doivent etre valides'
    },
    'Gestion de Position': {
        'Sizing': 'Base sur confiance (max 20%)',
        'Stop Loss': '5% par position',
        'Take Profit': 'Signal de sortie sentiment/technique'
    },
    'Signaux': {
        'Achat': 'Sentiment > 0.15 AND Prix > SMA AND RSI < 70',
        'Vente': 'Sentiment < -0.15 OR Prix < SMA OR RSI > 80'
    }
}

for category, details in components.items():
    print(f"\n{category}:")
    for key, value in details.items():
        print(f"  - {key}: {value}")

In [ ]:
# Tableau des dependances a installer

print("="*70)
print("DEPENDANCES PYTHON A INSTALLER")
print("="*70)

dependencies = [
    {
        'package': 'textblob',
        'install': 'pip install textblob',
        'usage': 'Analyse de sentiment basique',
        'note': 'Telecharger corpus: python -m textblob.download_corpora'
    },
    {
        'package': 'vaderSentiment',
        'install': 'pip install vaderSentiment',
        'usage': 'Analyse de sentiment specialise textes courts',
        'note': 'Recommande pour headlines et tweets'
    },
    {
        'package': 'newsapi-python',
        'install': 'pip install newsapi-python',
        'usage': 'Client Python pour NewsAPI',
        'note': 'Cle API gratuite: https://newsapi.org/'
    }
]

for dep in dependencies:
    print(f"\n{dep['package']}:")
    print(f"  Installation: {dep['install']}")
    print(f"  Usage: {dep['usage']}")
    print(f"  Note: {dep['note']}")

print("\n" + "="*70)
print("Dans QuantConnect, ces packages peuvent etre importes directement.")
print("="*70)

---

## Conclusion et Prochaines Etapes

### Recapitulatif

Dans ce notebook, nous avons couvert:

1. **Introduction au Sentiment** : Types de sentiment, sources de donnees, challenges

2. **Analyse avec Python** : TextBlob vs VADER, leurs avantages respectifs

3. **Integration QuantConnect** : Custom Data pour sentiment, Sentiment Alpha Model

4. **Strategie Contrarienne** : Trading contre les extremes de sentiment

5. **Approche Hybride** : Combiner sentiment + indicateurs techniques

6. **Strategie Complete** : News Sentiment Trading avec tous les composants

### Points Cles a Retenir

| Concept | Point Cle |
|---------|----------|
| **VADER vs TextBlob** | VADER meilleur pour textes courts et emphases |
| **Agregation** | Moyenne mobile lisse le bruit |
| **Confirmation** | Combiner sentiment + technique = plus robuste |
| **Contrarien** | Extremes de sentiment = signaux de reversal |
| **Sources** | NewsAPI gratuit pour prototypage |
| **Latence** | Le sentiment peut preceder ou suivre le prix |

### Limitations et Avertissements

| Limitation | Description |
|------------|-------------|
| **Bruit** | Beaucoup de news non-pertinentes |
| **Manipulation** | Fake news, pump & dump |
| **Latence** | Le temps de traitement peut etre critique |
| **Contexte** | VADER ne comprend pas le contexte financier specifique |
| **Sarcasme** | Difficile a detecter automatiquement |

### Ameliorations Possibles

1. **FinBERT** : Modele NLP specialise finance (Hugging Face)
2. **GPT pour sentiment** : LLM pour analyse plus fine
3. **Sentiment par secteur** : Agregation au niveau sectoriel
4. **Multi-source** : Combiner news + social + SEC filings
5. **Weighting par source** : Ponderer selon la fiabilite

### Ressources Complementaires

- [VADER Sentiment GitHub](https://github.com/cjhutto/vaderSentiment)
- [TextBlob Documentation](https://textblob.readthedocs.io/)
- [NewsAPI Documentation](https://newsapi.org/docs)
- [QuantConnect Custom Data](https://www.quantconnect.com/docs/v2/writing-algorithms/importing-data/streaming-data/custom-securities)
- [FinBERT (Hugging Face)](https://huggingface.co/ProsusAI/finbert)

---

**Notebook complete. L'analyse de sentiment peut etre un outil puissant, mais doit etre utilisee avec prudence et en combinaison avec d'autres signaux.**